# Finding Similar Songs - Part 2: Siamese Networks

In the first part of this tutorial I have introduced the traditional distance based approach to similarity estimations. The main idea is that features are extracted from the audio content. These features are numeric descriptions of semantically relevant information. An example for a high-level feature is the number of beats per minute which is a description for the tempo of a song. Music feature-sets are more abstract and describe the spectral or rhythmical distribution of energy. These are not single but vectors of numbers. Thus, a song is semantically described by this vector and if the set of extracted features spans over various music characteristics such as rhythm, timbre, harmonics, complexity, etc. then calculating the similarity of the vector's numbers is considered to be an approximation of music similarity. Thus, the lower the numerical distance between two vectors, the higher their acoustic similarity. For this reason these approaches are known as *Distance based* methods. They mainly depend on the selected sets of features and on the similarity metric chosen to compare their values.

In the second part of this tutorial we are now focussing on an approach, where the feature representation, as well as the similarity function is learned from the underlying dataset.


## Tutorial Overview

1. Loading data
2. Preprocess data
3. Define Model
4. Fit Model
5. Evaluate Model



## Requiremnts

The requirements are the same as for the first part of the tutorials. Please follow the instructions of part one if you have trouble running this tutorial.

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf

In [85]:
# visualization
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
from IPython.display import IFrame

# numeric and scientific processing
import numpy as np
import pandas as pd
#from scipy.interpolate import RectBivariateSpline

# misc
import os
import progressbar

In [86]:
from IPython.display import HTML, display

pd.set_option('display.max_colwidth', -1)

# Loading Data

We will use the same data that we downloaded from Spotify in Part 1 of the Tutorial. Because we used the joblib library, we will not have to wait that long, because the data is already cached on our harddrive.

In [87]:
DATASET_PATH    = "D:/Research/Data/MIR/MagnaTagATune"

METADATA_PATH   = "%s/metadata" % DATASET_PATH
AUTIO_ROOT_PATH = "%s/mp3_full" % DATASET_PATH

## Load Feature Data

In [88]:
with np.load("%s/ISMIR2018_tut_Magnagtagatune_spectrograms.npz" % DATASET_PATH) as npz:
    melspecs      = npz["features"]
    clip_id       = npz["clip_id"]
    
melspecs.shape

(6380, 80, 80)

## Load Metadata

In [89]:
metadata = pd.read_csv("%s/ismir2018_tut_part_2_genre_metadata.csv" % METADATA_PATH, index_col=0)

Align featuredata with metadata

In [90]:
feature_metadata = pd.DataFrame(np.arange(melspecs.shape[0]), 
                                columns = ["featurespace_id"])

feature_metadata["clip_id"] = clip_id

In [91]:
metadata = metadata.reset_index()
metadata = metadata.merge(feature_metadata, left_on="clip_id", right_on="clip_id", how="inner", left_index=True, right_index=False)
metadata = metadata.set_index("index")

Sort Metadata by Feature-space ID => metadata is aligned to feature-data

In [92]:
metadata.sort_values("featurespace_id", inplace=True)

Select feature-data for this metadata

In [93]:
melspecs = melspecs[metadata.featurespace_id].astype(np.float32)

In [94]:
melspecs = np.expand_dims(melspecs, 3)

In [95]:
metadata["featurespace_id"] = np.arange(metadata.shape[0])

In [96]:
metadata["player"] = '<audio src="' + \
                     metadata.mp3_path.str.replace(AUTIO_ROOT_PATH, "http://localhost:9999") + \
                     '" controls>'

## Load Labels

In [97]:
labels = pd.read_csv("%s/ismir2018_tut_part_2_genre_labels.csv" % METADATA_PATH, index_col=0)

In [98]:
labels.shape

(2617, 7)

In [99]:
labels = labels.loc[metadata.index]

# Siamese Networks

A Siamese neural network is a neural network architecture where two inputs are fed into the same stack of network layers. This is where the name comes from. The shared layers are "similar" to Siamese Twins. By feeding two inputs to the shared layers, two representations are generated which can be used for comparison. To train the network according a certain task, it requires labelled data. To learn a simlarity function, these labels should indicate if the two input are similar or dissimilar.

This is exactly the approach initially described by Hadsell-et-al.'06 (http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf). The authors create pairs of simlar and dissimilar images. These are fed into a Siamese NEtwork stack. Finally, the model calculates the eucledian distance between the two generated representations. A contrastive loss is used, to optimize the learned simlarity.

To calculate the similarity between a seed image and the rest of the collection, the model is applied to predict the distance between this seed image and every other. The result is a list of distances which has to be sorted descendingly.

The following code example follows this approach:

**Keras**

We use the high-level deep learning API Keras: https://keras.io/

In [100]:
from keras.models           import Model
from keras.layers           import Input, Lambda, Dense, Conv2D, Flatten, MaxPooling2D, Concatenate, Dropout, BatchNormalization, GaussianNoise
from keras.layers.merge     import concatenate
from keras.optimizers       import Nadam, SGD, Adam, RMSprop
from keras.constraints      import unit_norm
from keras.regularizers     import l2
from keras import backend as K

First we define a distance measure to compare the two representations. We will be using the well known Eucledian distance:

In [101]:
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))

### The Siamese Network Architecture

Now we define the Siamese Network Architecture. It consists of two fully connected layers. These layers are shared among the "Siamese twins". The network takes two inputs. One goes to the left twin, the other to the right one. The Eucledian distance of the output of each twin is calculated which is the final output of the model.

In [102]:
def create_siamese_network():

    input_ref = Input((80,80,1), name="input_ref")
    input_dif = Input((80,80,1), name="input_dif")
    
    gn = GaussianNoise(0.2)
    bn = BatchNormalization()

    cnn_a_1 = Conv2D(16, (21,11), padding="same", activation="relu", kernel_regularizer=l2(0.0001), name="conv_a")
    cnn_a_2 = MaxPooling2D((20,4),   name="max_pooling_a")
    cnn_a_3 = BatchNormalization()
    cnn_a_4 = Flatten(name="flatten_a")

    cnn_b_1 = Conv2D(16, (11,21), padding="same", activation="relu", kernel_regularizer=l2(0.0001), name="conv_b")
    cnn_b_2 = MaxPooling2D((4,20),   name="max_pooling_b")
    cnn_b_3 = BatchNormalization()
    cnn_b_4 = Flatten(name="flatten_b")

    mrg = Concatenate(axis=1, name="concatenate")

    hidden_layer = Dense(256, activation="elu", kernel_constraint=unit_norm())
    
    def get_shared_dnn(m_input):
        shared_cnn_a = cnn_a_4(cnn_a_3(cnn_a_2(cnn_a_1(bn(gn(m_input))))))
        shared_cnn_b = cnn_b_4(cnn_b_3(cnn_b_2(cnn_b_1(bn(gn(m_input))))))

        return hidden_layer(mrg([shared_cnn_a,shared_cnn_b]))

    siamese_ref = get_shared_dnn(input_ref)
    siamese_dif = get_shared_dnn(input_dif)

    dist = Lambda(euclidean_distance, output_shape=lambda x: x[0])([siamese_ref, siamese_dif])
    
    model = Model(inputs=[input_ref, input_dif], outputs=dist)
    
    return model

model = create_siamese_network().summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ref (InputLayer)          (None, 80, 80, 1)    0                                            
__________________________________________________________________________________________________
input_dif (InputLayer)          (None, 80, 80, 1)    0                                            
__________________________________________________________________________________________________
gaussian_noise_7 (GaussianNoise (None, 80, 80, 1)    0           input_ref[0][0]                  
                                                                 input_ref[0][0]                  
                                                                 input_dif[0][0]                  
                                                                 input_dif[0][0]                  
__________

**Contrastive Loss**

The contrastive loss is based on the Euclidean distance and measures the cost of data pairs. The objective of the contrastive loss is to minimize the distance between a similar pair and to separate any two dissimilar data with a distance margin

In [103]:
def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

### Create Data-Triplets

Now we have to prepare and partition the input data. Because it is a triplet-wise comparison approach, we have to create triplets of input instances with each row containing a referencec track, a similar example and a dissimilar example.

In [109]:
def create_pairs(feature_data, metadata, labels, num_triplets_per_track):
    
    data_ref  = []
    data_dif  = []
    gt_labels = []
    
    pbar = progressbar.ProgressBar(max_value=metadata.shape[0])
    
    i = 0
    
    for row_id, q_track in pbar(metadata.iterrows()):
        
        for _ in range(num_triplets_per_track):
            
            label_differences = np.abs(labels - labels.loc[row_id]).sum(axis=1)
            
            similar_instances    = label_differences[label_differences == 0]
            dissimilar_instances = label_differences[label_differences != 0] 
            
            # search similar and dissimilar examples
            pos_example_idx      = similar_instances.sample(1).index.values[0]
            neg_example_idx      = dissimilar_instances.sample(1).index.values[0]
            
            # create feature triplets
            feat_id_ref          = metadata.loc[row_id].featurespace_id
            feat_id_pos          = metadata.loc[pos_example_idx].featurespace_id
            feat_id_neg          = metadata.loc[neg_example_idx].featurespace_id
            
            #print([feat_id_ref, feat_id_pos, feat_id_neg])
            
            # genuine pair
            data_ref.append(feature_data[feat_id_ref])
            data_dif.append(feature_data[feat_id_pos])
            gt_labels.append(1)
            
            data_ref.append(feature_data[feat_id_ref])
            data_dif.append(feature_data[feat_id_neg])
            gt_labels.append(0)
            
        #i += 1
        
        #if i > 10:
        #    break

    return [np.asarray(data_ref), np.asarray(data_dif)], np.asarray(gt_labels)

#data_pairs, paired_labels = create_pairs(melspecs, metadata, labels, 1)

Exectue the function to prepare the input data

In [110]:
# create pairs
data_pairs, paired_labels = create_pairs(melspecs, metadata, labels, 5)

# check - how many instances have we created?
data_pairs[0].shape

100% (2617 of 2617) |####################| Elapsed Time: 0:01:03 Time:  0:01:03


(26170, 80, 80, 1)

Prepare the Siamese Neural Network

In [111]:
# define the model
model_orig = create_siamese_network()

# define the optimizer
opt = Adam(lr=0.0001, decay=0.001)

# compile the model
model_orig.compile(loss      = contrastive_loss, 
                   optimizer = opt)

Train the network

In [112]:
model_orig.fit([data_pairs[0][:-2000], data_pairs[1][:-2000]], 
                paired_labels[:-2000], 
                batch_size       = 64, 
                verbose          = 1, 
                epochs           = 10,
                shuffle          = False, # important !
                validation_data = [[data_pairs[0][-2000:], data_pairs[1][-2000:]], paired_labels[-2000:]]);

Train on 24170 samples, validate on 2000 samples
Epoch 1/10
24170/24170 [==============================] - 27s 1ms/step - loss: 3.9747 - val_loss: 24.0707
Epoch 2/10
24170/24170 [==============================] - 23s 967us/step - loss: 0.4863 - val_loss: 0.9956
Epoch 3/10
24170/24170 [==============================] - 23s 967us/step - loss: 0.3652 - val_loss: 0.3153
Epoch 4/10
24170/24170 [==============================] - 23s 967us/step - loss: 0.3089 - val_loss: 0.3826
Epoch 5/10
24170/24170 [==============================] - 23s 964us/step - loss: 0.2791 - val_loss: 0.3523
Epoch 6/10
24170/24170 [==============================] - 23s 962us/step - loss: 0.2610 - val_loss: 0.3303
Epoch 7/10
24170/24170 [==============================] - 23s 960us/step - loss: 0.2487 - val_loss: 0.3197
Epoch 8/10
24170/24170 [==============================] - 23s 965us/step - loss: 0.2394 - val_loss: 0.3652
Epoch 9/10
24170/24170 [==============================] - 23s 948us/step - loss: 0.2319 - val_lo

# Evaluate

Now that we have a trained model, we want to evaluate its performance. We will first play around with some examples, listen to the results and judge by our subjective interpretation before we persue a general evaluation.

### Evaluate by Example

The following function calculated the distances between a given query track and all other tracks of the collection. The result is a list of distances where the smallest distance coresponds with the most similar track. The list is sorted descendingly and the top-ten similar tracks are presented below the information of the query track. The Spotify playlist we created at the beginning will also be updated with the query results. Thus, you can listen to it in your Spotify client.

In [113]:
def similar(model, query_idx):
    

    print(metadata.iloc[query_idx])
    
    ref_feat = melspecs[[metadata.iloc[query_idx].featurespace_id]]
    ref_feat = np.repeat(ref_feat, melspecs.shape[0], axis=0)
    
    # calclulate predicted distances between query track and all others
    res = model.predict([ref_feat, melspecs])

    # reshape
    res = res.flatten()
    
    # get sorted indexes in ascending order (smallest distance to query track first)
    si = np.argsort(res)
    
    # output filter
    display_cols = ["artist", "title", "album", "player"]
    
    
    
    return HTML(metadata.set_index("featurespace_id").loc[si[:11]][display_cols].to_html(escape=False))

Now let's check the results for some individual tracks by supplying the index to our dataset:

In [151]:
similar(model_orig, 311)

clip_id            50892                                                                                                                                                                  
mp3_path           D:/Research/Data/MIR/MagnaTagATune/mp3_full/6/doc_rossi-demarzi6_sonatas_for_cetra_o_kitara-13-the_fairy_hornpipe__whisky_youre_the_devil_trad__arr__rossi-117-146.mp3 
track_number       13                                                                                                                                                                     
title              The Fairy Hornpipe - Whisky You're the Devil (trad. arr. Rossi)                                                                                                        
artist             Doc Rossi                                                                                                                                                              
album              Demarzi-6 Sonatas for Cetra o Kitara          

,artist,title,album,player
featurespace_id,,,,
311,Doc Rossi,The Fairy Hornpipe - Whisky You're the Devil (trad. arr. Rossi),Demarzi-6 Sonatas for Cetra o Kitara,
943,Magnatune Remixed,Sonata in G Harp Minor - Fourstones,Ridin the Faders 2,
338,Jacob Heringman,The Hay,Blame Not My Lute,
832,William Brooks,Convertible Girls Live,Buffalo Treason,
628,Doc Rossi,Sonata III Grazioso,Demarzi-6 Sonatas for Cetra o Kitara,
81,Philharmonia Baroque,Entree tres gaye des Troubadours _ Air tres gay _ Gavotte _ Menuet (Rameau_ Les Paladins),Rameau and Leclair,
1002,Trip Wamsley,The Deeper Thing (for Sam Lewis),Its Better This Way,
458,Oberlin Consort of Viols,Set a 6 in G minor,5 and 6 part Fantasies of William Lawes,
2211,The Bots,Push,Now is The Time,


# Improve Performance through prior Knowledge

The original approach only uses genuine and impostor pairs and does not consider any further prior knowledge. In that sense if two tracks belong to the same playlist, they are considered similar, if not, than they are not. But, because we have chosen genre-playlists, there are genres that are more similar than others. This is of course highly subjective and depends on the listening behaviour and experience of a listener.

The following list represents my own interpretation of genre similarities:

In [118]:
sim = [[["blues",      "blues"],     1.0],
       [["blues",      "classical"], 0.0],
       [["blues",      "country"],   0.8],
       [["blues",      "jazz"],      0.3],
       [["blues",      "pop"],       0.0],
       [["blues",      "rock"],      0.1],
       [["blues",      "techno"],    0.0],
       [["classical",  "classical"], 1.0],
       [["classical",  "country"],   0.0],
       [["classical",  "jazz"],      0.0],
       [["classical",  "pop"],       0.0],
       [["classical",  "rock"],      0.0],
       [["classical",  "techno"],    0.0],
       [["country",    "country"],   1.0],
       [["country",    "jazz"],      0.1],
       [["country",    "pop"],       0.2],
       [["country",    "rock"],      0.3],
       [["country",    "techno"],    0.0],
       [["jazz",       "jazz"],      1.0],
       [["jazz",       "pop"],       0.0],
       [["jazz",       "rock"],      0.1],
       [["jazz",       "techno"],    0.0],
       [["pop",        "pop"],       1.0],
       [["pop",        "rock"],      0.2],
       [["pop",        "techno"],    0.8],
       [["rock",       "rock"],      1.0],
       [["rock",       "techno"],    0.0],
       [["techno",     "techno"],    1.0]]

The following code creates a symmetric lookup-table from the list above:

In [125]:
# get all playlist-names from our dataset
playlist_names = [pl for pl in labels.columns]

# create the lookup-table
playlist_similarities = pd.DataFrame(np.zeros((len(labels.columns),len(labels.columns))), 
                                     index   = playlist_names, 
                                     columns = playlist_names)

# self-similarity
for i in range(len(playlist_names)):
    for j in range(len(playlist_names)):
        if i == j:
            playlist_similarities.iloc[i,j] = 1.0

# genre-similarities
for s in sim:
    playlist_similarities.loc[s[0][0],s[0][1]] = s[1]
    playlist_similarities.loc[s[0][1],s[0][0]] = s[1]

# show results
playlist_similarities

,blues,classical,country,jazz,pop,rock,techno
blues,1.0,0.0,0.8,0.3,0.0,0.1,0.0
classical,0.0,1.0,0.0,0.0,0.0,0.0,0.0
country,0.8,0.0,1.0,0.1,0.2,0.3,0.0
jazz,0.3,0.0,0.1,1.0,0.0,0.1,0.0
pop,0.0,0.0,0.2,0.0,1.0,0.2,0.8
rock,0.1,0.0,0.3,0.1,0.2,1.0,0.0
techno,0.0,0.0,0.0,0.0,0.8,0.0,1.0


In [133]:
labels.loc[0].idxmax(axis=1)

'classical'

In [140]:
def create_pairs_with_sims(feature_data, metadata, labels, playlist_similarities, num_triplets_per_track):
    
    data_ref  = []
    data_dif  = []
    gt_labels = []
    
    pbar = progressbar.ProgressBar(max_value=metadata.shape[0])
    
    i = 0
    
    for row_id, q_track in pbar(metadata.iterrows()):
        
        for _ in range(num_triplets_per_track):
            
            label_differences = np.abs(labels - labels.loc[row_id]).sum(axis=1)
            
            similar_instances    = label_differences[label_differences == 0]
            dissimilar_instances = label_differences[label_differences != 0] 
            
            # search similar and dissimilar examples
            pos_example_idx      = similar_instances.sample(1).index.values[0]
            neg_example_idx      = dissimilar_instances.sample(1).index.values[0]
            
            # create feature triplets
            feat_id_ref          = metadata.loc[row_id].featurespace_id
            feat_id_pos          = metadata.loc[pos_example_idx].featurespace_id
            feat_id_neg          = metadata.loc[neg_example_idx].featurespace_id
            
            #print([feat_id_ref, feat_id_pos, feat_id_neg])
            
            # genuine pair
            data_ref.append(feature_data[feat_id_ref])
            data_dif.append(feature_data[feat_id_pos])
            
            label = playlist_similarities.loc[labels.loc[row_id].idxmax(axis=1), 
                                              labels.loc[pos_example_idx].idxmax(axis=1)]
                        
            gt_labels.append(label)
            
            data_ref.append(feature_data[feat_id_ref])
            data_dif.append(feature_data[feat_id_neg])
            
            label = playlist_similarities.loc[labels.loc[row_id].idxmax(axis=1), 
                                              labels.loc[neg_example_idx].idxmax(axis=1)]
            
            gt_labels.append(label)
            
        #i += 1
        #
        #if i > 10:
        #    break

    return [np.asarray(data_ref), np.asarray(data_dif)], np.asarray(gt_labels)

#data_pairs, paired_labels = create_pairs_with_sims(melspecs, metadata, labels, playlist_similarities, 1)

### Train network with prior knowledge

With this lookup-table we can create more accurate input pairs. Insted of similar/dissimilar we can now apply the supplied similarites:

Execute the function to prepare the data:

In [141]:
# get pairs
data_pairs, paired_labels = create_pairs_with_sims(melspecs, metadata, labels, playlist_similarities, 5)

# check - how many instances have we created?
data_pairs[0].shape

100% (2617 of 2617) |####################| Elapsed Time: 0:01:24 Time:  0:01:24


AttributeError: 'list' object has no attribute 'shape'

Prepare the Siamese Neural Network

In [143]:
# define the model
model_sim = create_siamese_network()

# define the optimizer
opt = Adam(lr=0.0001, decay=0.001)

# compile the model
model_sim.compile(loss      = contrastive_loss, 
                  optimizer = opt)

Train the model on the adapted data:

In [144]:
model_sim.fit([data_pairs[0][:-2000], data_pairs[1][:-2000]], 
                paired_labels[:-2000], 
                batch_size       = 64, 
                verbose          = 1, 
                epochs           = 10,
                shuffle          = False, # important !
                validation_data = [[data_pairs[0][-2000:], data_pairs[1][-2000:]], paired_labels[-2000:]]);

Train on 24170 samples, validate on 2000 samples
Epoch 1/10
24170/24170 [==============================] - 27s 1ms/step - loss: 4.5618 - val_loss: 5.0875
Epoch 2/10
24170/24170 [==============================] - 23s 942us/step - loss: 0.4973 - val_loss: 355.1346
Epoch 3/10
24170/24170 [==============================] - 23s 961us/step - loss: 0.3865 - val_loss: 0.3822
Epoch 4/10
24170/24170 [==============================] - 23s 972us/step - loss: 0.3354 - val_loss: 0.4302
Epoch 5/10
24170/24170 [==============================] - 23s 970us/step - loss: 0.3061 - val_loss: 0.3025
Epoch 6/10
24170/24170 [==============================] - 23s 967us/step - loss: 0.2853 - val_loss: 708.8054
Epoch 7/10
24170/24170 [==============================] - 23s 968us/step - loss: 0.2672 - val_loss: 0.3878
Epoch 8/10
24170/24170 [==============================] - 23s 960us/step - loss: 0.2517 - val_loss: 1.2015
Epoch 9/10
24170/24170 [==============================] - 23s 970us/step - loss: 0.2402 - val

Now let's check the results for some individual tracks by supplying the index to our dataset:

In [152]:
similar(model_sim, 311)

clip_id            50892                                                                                                                                                                  
mp3_path           D:/Research/Data/MIR/MagnaTagATune/mp3_full/6/doc_rossi-demarzi6_sonatas_for_cetra_o_kitara-13-the_fairy_hornpipe__whisky_youre_the_devil_trad__arr__rossi-117-146.mp3 
track_number       13                                                                                                                                                                     
title              The Fairy Hornpipe - Whisky You're the Devil (trad. arr. Rossi)                                                                                                        
artist             Doc Rossi                                                                                                                                                              
album              Demarzi-6 Sonatas for Cetra o Kitara          

,artist,title,album,player
featurespace_id,,,,
311,Doc Rossi,The Fairy Hornpipe - Whisky You're the Devil (trad. arr. Rossi),Demarzi-6 Sonatas for Cetra o Kitara,
651,Paul Beier,Aria I affettuosa,Alessandro Piccinini,
1507,Heavy Mellow,Open Sea,Horizons,
1650,C Layne,Vertigo Blues,The Sun Will Come Out to Blind You,
2203,ABA STRUCTURE,The Internal,EPIC,
1340,Lisa DeBenedictis,Chimney Sweeping,Fruitless,
452,Barry Phillips,Bjernulf's brollopolska,Trad,
2231,Ambient Teknology,Goa Life,Phoenix,
794,Doc Rossi,The Fairy Hornpipe - Whisky You're the Devil (trad. arr. Rossi),Demarzi-6 Sonatas for Cetra o Kitara,


# Improve Performance through Identity

So far we have taught the network what is similar and what not, but we have not shown it, what is identical. All input pairs created so far missed to pass identical data. In the following step, we will include identical pairs into the training instances. To emphasis the identity, only identical pairs will be assigned a label of 1. All other similarity values of the lookup-table will be decreased by 0.1. Thus, tracks of the same playlist will have a similarity value 0f 0.9.

In [154]:
def create_pairs_with_sims_and_identity(feature_data, metadata, labels, playlist_similarities, num_triplets_per_track):
    
    data_ref  = []
    data_dif  = []
    gt_labels = []
    
    pbar = progressbar.ProgressBar(max_value=metadata.shape[0])
    
    i = 0
    
    for row_id, q_track in pbar(metadata.iterrows()):
        
        for j in range(num_triplets_per_track):
            
            label_differences = np.abs(labels - labels.loc[row_id]).sum(axis=1)
            
            similar_instances    = label_differences[label_differences == 0]
            dissimilar_instances = label_differences[label_differences != 0] 
            
            # search similar and dissimilar examples
            pos_example_idx      = similar_instances.sample(1).index.values[0]
            neg_example_idx      = dissimilar_instances.sample(1).index.values[0]
            
            # create feature triplets
            feat_id_ref          = metadata.loc[row_id].featurespace_id
            feat_id_pos          = metadata.loc[pos_example_idx].featurespace_id
            feat_id_neg          = metadata.loc[neg_example_idx].featurespace_id
            
            #print([feat_id_ref, feat_id_pos, feat_id_neg])
            
            if j == 0:
                
                data_ref.append(feature_data[feat_id_ref])
                data_dif.append(feature_data[feat_id_ref])
                gt_labels.append(1)
                
                data_ref.append(feature_data[feat_id_ref])
                data_dif.append(feature_data[feat_id_neg])
                
                label = playlist_similarities.loc[labels.loc[row_id].idxmax(axis=1), 
                                                  labels.loc[neg_example_idx].idxmax(axis=1)]
                gt_labels.append(label)
            
            # genuine pair
            data_ref.append(feature_data[feat_id_ref])
            data_dif.append(feature_data[feat_id_pos])
            
            label = playlist_similarities.loc[labels.loc[row_id].idxmax(axis=1), 
                                              labels.loc[pos_example_idx].idxmax(axis=1)]
                        
            gt_labels.append(label)
            
            data_ref.append(feature_data[feat_id_ref])
            data_dif.append(feature_data[feat_id_neg])
            
            label = playlist_similarities.loc[labels.loc[row_id].idxmax(axis=1), 
                                              labels.loc[neg_example_idx].idxmax(axis=1)]
            
            gt_labels.append(label)
            
        #i += 1
        #
        #if i > 10:
        #    break

    return [np.asarray(data_ref), np.asarray(data_dif)], np.asarray(gt_labels)

#data_pairs, paired_labels = create_pairs_with_sims_and_identity(melspecs, metadata, labels, playlist_similarities, 1)

In [157]:
playlist_similarities[playlist_similarities == 1] = 0.9
playlist_similarities

,blues,classical,country,jazz,pop,rock,techno
blues,0.9,0.0,0.8,0.3,0.0,0.1,0.0
classical,0.0,0.9,0.0,0.0,0.0,0.0,0.0
country,0.8,0.0,0.9,0.1,0.2,0.3,0.0
jazz,0.3,0.0,0.1,0.9,0.0,0.1,0.0
pop,0.0,0.0,0.2,0.0,0.9,0.2,0.8
rock,0.1,0.0,0.3,0.1,0.2,0.9,0.0
techno,0.0,0.0,0.0,0.0,0.8,0.0,0.9


Execute the function to prepare the data:

In [158]:
data_pairs, paired_labels = create_pairs_with_sims_and_identity(melspecs, metadata, labels, playlist_similarities, 5)

100% (2617 of 2617) |####################| Elapsed Time: 0:01:16 Time:  0:01:16


Prepare the Neural Network

In [159]:
# define the model
model_sim_id = create_siamese_network()

# define the optimizer
opt = Adam(lr=0.0001, decay=0.001)

# compile the model
model_sim_id.compile(loss      = contrastive_loss, 
                  optimizer = opt)

Train the model on the adapted data

In [160]:
model_sim_id.fit([data_pairs[0][:-2000], data_pairs[1][:-2000]], 
                  paired_labels[:-2000], 
                  batch_size       = 64, 
                  verbose          = 1, 
                  epochs           = 10,
                  shuffle          = False, # important !
                  validation_data = [[data_pairs[0][-2000:], data_pairs[1][-2000:]], paired_labels[-2000:]]);

Train on 29404 samples, validate on 2000 samples
Epoch 1/10
29404/29404 [==============================] - 32s 1ms/step - loss: 2.6508 - val_loss: 104.3290
Epoch 2/10
29404/29404 [==============================] - 28s 962us/step - loss: 0.3760 - val_loss: 244.7066
Epoch 3/10
29404/29404 [==============================] - 28s 966us/step - loss: 0.2911 - val_loss: 0.4295
Epoch 4/10
29404/29404 [==============================] - 28s 959us/step - loss: 0.2555 - val_loss: 0.9778
Epoch 5/10
29404/29404 [==============================] - 28s 967us/step - loss: 0.2359 - val_loss: 0.2540
Epoch 6/10
29404/29404 [==============================] - 28s 966us/step - loss: 0.2228 - val_loss: 0.4630
Epoch 7/10
29404/29404 [==============================] - 28s 967us/step - loss: 0.2133 - val_loss: 0.2799
Epoch 8/10
29404/29404 [==============================] - 29s 970us/step - loss: 0.2060 - val_loss: 0.2244
Epoch 9/10
29404/29404 [==============================] - 28s 964us/step - loss: 0.2000 - val

Now let's check the results for some individual tracks by supplying the index to our dataset:

In [161]:
similar(model_sim_id, 311)

clip_id            50892                                                                                                                                                                  
mp3_path           D:/Research/Data/MIR/MagnaTagATune/mp3_full/6/doc_rossi-demarzi6_sonatas_for_cetra_o_kitara-13-the_fairy_hornpipe__whisky_youre_the_devil_trad__arr__rossi-117-146.mp3 
track_number       13                                                                                                                                                                     
title              The Fairy Hornpipe - Whisky You're the Devil (trad. arr. Rossi)                                                                                                        
artist             Doc Rossi                                                                                                                                                              
album              Demarzi-6 Sonatas for Cetra o Kitara          

,artist,title,album,player
featurespace_id,,,,
311,Doc Rossi,The Fairy Hornpipe - Whisky You're the Devil (trad. arr. Rossi),Demarzi-6 Sonatas for Cetra o Kitara,
794,Doc Rossi,The Fairy Hornpipe - Whisky You're the Devil (trad. arr. Rossi),Demarzi-6 Sonatas for Cetra o Kitara,
512,Paul Beier,Sonate in D minor_ Toccata in tre parti Gagliarda Gagliarda Volta,Michelagnolo Galilei,
165,Sherefe,Geldim Bir Kara Tasha,Opium,
380,John Williams,Country Kitchen,Dusty Porch,
476,Tanya Tomkins and Eric Zivian,Sonata in D Major op. 102 no. 2 Allegro con brio,Beethoven Piano and Cello Works,
799,William Brooks,Grace,Fowl Mouth,
65,Paul Beier,Ung gaij bergier - Canzone Francese a quattro di Thomas Crequillon,Simone Molinaro,
458,Oberlin Consort of Viols,Set a 6 in G minor,5 and 6 part Fantasies of William Lawes,
